In [1]:
import pandas as pd
import numpy as np

In [2]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_651a44c6256d4289950e25263ac926ad = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_651a44c6256d4289950e25263ac926ad = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_651a44c6256d4289950e25263ac926ad = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='QaG9CCaAbuUWvmoq8UTBa8Oh9b4PM-yFULVsGptC0J94',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_651a44c6256d4289950e25263ac926ad)

body = client_651a44c6256d4289950e25263ac926ad.get_object(Bucket='waterqualitydrinkingwaterpotabili-donotdelete-pr-y19ucaazm8kmih',Key='Waterqualitypreprocess.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)
data.head()


,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,7.371914,148.193698,42059.380417,7.966710,324.546262,544.848432,17.166504,62.677756,4.338957,1
1,3.445062,207.926260,33424.768678,8.782147,384.007006,441.785876,13.805902,30.284597,4.184397,0
2,7.475537,215.429102,31359.386456,7.498495,337.462379,477.235053,12.186310,53.908820,4.015574,0
3,7.494212,210.347276,29252.501112,5.922704,339.846577,445.061231,16.917113,84.868548,5.492705,0
4,6.168839,238.301048,24827.516641,7.720011,269.490173,482.673327,22.091990,79.169281,4.435760,0


In [3]:
data.Potability.value_counts()

1    1200
0    1200
Name: Potability, dtype: int64

In [4]:
x = data.drop(['Potability'], axis = 1)
y = data['Potability']

Applying StandardScaler befor fitting ML model to normalize the features.

In [5]:
from sklearn.preprocessing import StandardScaler
st = StandardScaler()
col= x.columns
x[col] = st.fit_transform(x[col])
x[col]

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
0,0.191413,-1.446535,2.243012,0.522934,-0.184372,1.428375,0.837561,-0.262077,0.466215
1,-2.327174,0.362590,1.273805,1.022431,1.221037,0.166751,-0.180652,-2.287969,0.267959
2,0.257874,0.589829,1.041973,0.236127,0.120912,0.600696,-0.671365,-0.810493,0.051410
3,0.269852,0.435916,0.805482,-0.729126,0.177265,0.206846,0.761999,1.125752,1.946132
4,-0.580210,1.282554,0.308792,0.371817,-1.485673,0.667268,2.329911,0.769316,0.590384
...,...,...,...,...,...,...,...,...,...
2395,0.861120,0.374698,0.691260,0.411089,-0.827641,0.200711,-0.532336,0.743502,1.224455
2396,0.835588,1.580331,0.645158,-0.007651,0.253170,0.064707,1.289731,-0.848333,2.217113
2397,1.221403,0.092128,-0.207754,0.279633,0.480733,0.782637,1.581517,-0.168298,-0.994731
2398,1.108536,0.286314,-1.005207,-0.251910,-1.627664,-0.353535,1.365522,0.068488,0.071833


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x,y, test_size = 0.1)

In [9]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(X_train,Y_train)
y_pred=classifier.predict(X_test)

In [10]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Y_test, y_pred))

Accuracy: 0.8583333333333333


In [11]:
random_grid = {'bootstrap': [True, False],
               'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
               'max_features': ['auto', 'sqrt'],
               'min_samples_leaf': [1, 2, 4],
               'min_samples_split': [2, 5, 10],
               'n_estimators': [130, 180, 230]}

In [12]:
from sklearn.model_selection import RandomizedSearchCV
# Use the random grid to search for best hyperparameters
# First create the base model to tune
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=7, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train,Y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] n_estimators=230, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=90, bootstrap=True 


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=230, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=90, bootstrap=True, total=   2.2s
[CV] n_estimators=230, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=90, bootstrap=True 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    2.2s remaining:    0.0s


[CV]  n_estimators=230, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=90, bootstrap=True, total=   2.1s
[CV] n_estimators=230, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=90, bootstrap=True 
[CV]  n_estimators=230, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=90, bootstrap=True, total=   2.1s
[CV] n_estimators=180, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=90, bootstrap=False 
[CV]  n_estimators=180, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=90, bootstrap=False, total=   2.7s
[CV] n_estimators=180, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=90, bootstrap=False 
[CV]  n_estimators=180, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=90, bootstrap=False, total=   2.4s
[CV] n_estimators=180, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=90, bootstrap=False 
[CV]  n_estimators=180, mi

...

[CV]  n_estimators=130, min_samples_split=10, min_samples_leaf=1, max_features=auto, max_depth=70, bootstrap=True, total=   1.2s
[CV] n_estimators=130, min_samples_split=10, min_samples_leaf=1, max_features=auto, max_depth=70, bootstrap=True 
[CV]  n_estimators=130, min_samples_split=10, min_samples_leaf=1, max_features=auto, max_depth=70, bootstrap=True, total=   1.2s
[CV] n_estimators=180, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=100, bootstrap=False 
[CV]  n_estimators=180, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=100, bootstrap=False, total=   1.1s
[CV] n_estimators=180, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=100, bootstrap=False 
[CV]  n_estimators=180, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=100, bootstrap=False, total=   1.1s
[CV] n_estimators=180, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=100, bootstrap=False 
[CV]  n_estimators=180,

[CV]  n_estimators=180, min_samples_split=2, min_samples_leaf=1, max_features=sqrt, max_depth=60, bootstrap=False, total=   1.2s
[CV] n_estimators=230, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=None, bootstrap=False 
[CV]  n_estimators=230, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=None, bootstrap=False, total=   3.5s
[CV] n_estimators=230, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=None, bootstrap=False 
[CV]  n_estimators=230, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=None, bootstrap=False, total=   3.6s
[CV] n_estimators=230, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=None, bootstrap=False 
[CV]  n_estimators=230, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=None, bootstrap=False, total=   3.3s
[CV] n_estimators=230, min_samples_split=2, min_samples_leaf=1, max_features=auto, max_depth=30, bootstrap=True 
[CV]  n_est

[CV]  n_estimators=230, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=20, bootstrap=True, total=   2.3s
[CV] n_estimators=230, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=20, bootstrap=True 
[CV]  n_estimators=230, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=20, bootstrap=True, total=   2.2s
[CV] n_estimators=230, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=20, bootstrap=True 
[CV]  n_estimators=230, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=20, bootstrap=True, total=   2.2s
[CV] n_estimators=130, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=90, bootstrap=False 
[CV]  n_estimators=130, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=90, bootstrap=False, total=   2.0s
[CV] n_estimators=130, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=90, bootstrap=False 
[CV]  n_estimators=130, min_sampl

[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  7.6min finished


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [130, 180, 230]},
                   random_state=7, verbose=2)

In [13]:
rf_random.best_params_

{'n_estimators': 230,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 60,
 'bootstrap': False}

In [15]:
rf_random.best_estimator_

RandomForestRegressor(bootstrap=False, max_depth=60, max_features='sqrt',
                      n_estimators=230)

In [28]:
y_pred_r = rf_random.predict(X_test)
y_pred_r = y_pred_r.round()

In [29]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Y_test, y_pred_r))

Accuracy: 0.8791666666666667


In [30]:
!pip install -U ibm-watson-machine-learning

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 1.8 MB 15.1 MB/s eta 0:00:01
  Attempting uninstall: ibm-watson-machine-learning
    Found existing installation: ibm-watson-machine-learning 1.0.173
    Uninstalling ibm-watson-machine-learning-1.0.173:
      Successfully uninstalled ibm-watson-machine-learning-1.0.173


In [34]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "2BUeG0Dyug3OIM-iJDYqZoOqzy0YCLKYxlzCZPrH8JZ4"
}
client = APIClient(wml_credentials)
print(client.version)

1.0.173


In [35]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [36]:
space_uid = guid_from_space_name(client, 'waterquality')
print("Space UID = " + space_uid)

Space UID = 365a9297-2b23-458b-ad22-4b95df5fae64


In [37]:
client.set.default_space(space_uid)

'SUCCESS'

In [38]:
sofware_spec_uid = client.software_specifications.get_id_by_name("default_py3.8")
metadata = {
            client.repository.ModelMetaNames.NAME: 'waterqualitydeploy',
            client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.23',
            client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

published_model = client.repository.store_model(
    model=rf_random,
    meta_props=metadata)

In [39]:
import json

published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

{
  "entity": {
    "software_spec": {
      "id": "ab9e1b80-f2ce-592c-a7d2-4f2344f77194",
      "name": "default_py3.8"
    },
    "type": "scikit-learn_0.23"
  },
  "metadata": {
    "created_at": "2021-12-07T07:51:19.107Z",
    "id": "c29df92a-ce7f-4c20-92b9-c9bb6bd94a59",
    "modified_at": "2021-12-07T07:51:27.656Z",
    "name": "waterqualitydeploy",
    "owner": "IBMid-550009MFNF",
    "resource_key": "20bc62e4-b48f-4d27-ae1c-c0b814549e93",
    "space_id": "365a9297-2b23-458b-ad22-4b95df5fae64"
  },
  "system": {
    "warnings": []
  }
}


In [40]:
client.repository.list_models()

------------------------------------  ------------------  ------------------------  -----------------
ID                                    NAME                CREATED                   TYPE
c29df92a-ce7f-4c20-92b9-c9bb6bd94a59  waterqualitydeploy  2021-12-07T07:51:19.002Z  scikit-learn_0.23
------------------------------------  ------------------  ------------------------  -----------------


In [41]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "waterqualitydeploy",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)



#######################################################################################

Synchronous deployment creation for uid: 'c29df92a-ce7f-4c20-92b9-c9bb6bd94a59' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='5a40960c-2088-47cd-96ab-97baf5e729da'
------------------------------------------------------------------------------------------------




In [42]:
deployment_uid = client.deployments.get_uid(created_deployment)
client.deployments.get_details(deployment_uid)

{'entity': {'asset': {'id': 'c29df92a-ce7f-4c20-92b9-c9bb6bd94a59'},
  'custom': {},
  'deployed_asset_type': 'model',
  'hardware_spec': {'id': 'Not_Applicable', 'name': 'S', 'num_nodes': 1},
  'name': 'waterqualitydeploy',
  'online': {},
  'space_id': '365a9297-2b23-458b-ad22-4b95df5fae64',
  'status': {'online_url': {'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/5a40960c-2088-47cd-96ab-97baf5e729da/predictions'},
   'serving_urls': ['https://us-south.ml.cloud.ibm.com/ml/v4/deployments/5a40960c-2088-47cd-96ab-97baf5e729da/predictions'],
   'state': 'ready'}},
 'metadata': {'created_at': '2021-12-07T07:52:26.003Z',
  'id': '5a40960c-2088-47cd-96ab-97baf5e729da',
  'modified_at': '2021-12-07T07:52:26.003Z',
  'name': 'waterqualitydeploy',
  'owner': 'IBMid-550009MFNF',
  'space_id': '365a9297-2b23-458b-ad22-4b95df5fae64'},
 'system': {'warnings': [{'id': 'Deprecated',
    'message': 'online_url is deprecated and will be removed in a future release. Use serving_urls inste

In [43]:
scoring_endpoint = client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

https://us-south.ml.cloud.ibm.com/ml/v4/deployments/5a40960c-2088-47cd-96ab-97baf5e729da/predictions


In [44]:
# use our WML client to score our model
# add some test data
scoring_payload = {"input_data": [
    {'fields': ['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity',
                'Organic_carbon', 'Trihalomethanes', 'Turbidity'], 
     'values': [[6, 191, 6200, 7.5, 333, 425, 20.8, 98.3, 3.7]]
    }]}

In [45]:
predictions = client.deployments.score(deployment_uid, scoring_payload)
print('prediction',json.dumps(predictions, indent=2))

prediction {
  "predictions": [
    {
      "fields": [
        "prediction"
      ],
      "values": [
        [
          0.45217391304347826
        ]
      ]
    }
  ]
}
